The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

1\.d Find out which is the letter with most counts.

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

In [ ]:
#!/bin/bash


# 1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from 
# [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. 
# First check whether the file is already there
cd $HOME
mkdir -p students

filename=LCP_22-23_students.csv

cd students

if [ ! -f "./$filename" ]
then
    wget https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv
else
    echo "$filename is already in ./students"
fi


# 1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.
touch PoD.txt
grep "PoD" $filename > PoD.txt
touch Physics.txt
grep "Physics" $filename > Physics.txt
echo "PoD.txt and Physics.txt created successfully"


# 1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 
echo "Number of Surname that starts with:"
for letter in {A..Z}; do
    count=$(grep -v "Family" "$filename" | grep -c "^$letter")
    echo "$letter = $count";
done


# 1\.d Find out which is the letter with most counts.
bestletter=A
bestcounts=0
for letter in {A..Z}; do
    count=$(grep -v "Family" "$filename" | grep -c "^$letter")
    if [ $count -gt $bestcounts ]; then 
        bestletter=$letter 
        bestcounts=$count;
    fi
done
echo "The letter with most counts is $bestletter ($bestcounts counts)"


# 1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students 
# "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file.

for i in {0..17}; do touch group$i.txt; done
for i in {0..17}; do > group$i.txt; done    # clean input

number_students=$(grep -c -v "Family" "$filename")
maxindex=$(( $number_students + 1 ))
for (( index=2; index<=$maxindex; index++ )); do
    student_number=$(( $index - 1 ))
    numbergroup=$(( $student_number % 18 )) 
    student=$(sed -n "${index}p" "$filename")
    echo "$student" >> group$numbergroup.txt;
done

### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

2\.b How many even numbers are there?

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

In [ ]:
#!/bin/bash

# 2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`
cd $HOME

filename=./LaboratoryOfComputationalPhysics_Y8/data.csv
cpfilename=./LaboratoryOfComputationalPhysics_Y8/data.txt

touch $cpfilename
echo "$( sed "/#/d" "$filename" | sed "s/,//g" )" > $cpfilename 


# 2\.b How many even numbers are there?
echo "There are $(grep -o "[0-9]\+" "$cpfilename" | grep "[02468]$" | wc -l) even number in $cpfilename"

# otherwise...
count=0
for num in $(cat "$cpfilename"); do     # cat = concatenate (load the file in memory)
    if (( num % 2 == 0 )); then
        ((count++))
    fi
done
echo $count


# 2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`.
# Count the entries of each of the two groups.
counter=0
limit=$( echo "scale=6;100*sqrt(3)/2" | bc )
Nlines=$( grep -c "^" "$cpfilename" )
for (( i=1; i<=$Nlines; i++ )); do
    a=$( sed -n "${i}p" "$cpfilename" | cut -d ' ' -f 1 )
    b=$( sed -n "${i}p" "$cpfilename" | cut -d ' ' -f 2 )
    c=$( sed -n "${i}p" "$cpfilename" | cut -d ' ' -f 3 )
    distance=$( echo "scale=6;sqrt($a^2 + $b^2 + $c^2)" | bc )
    check=$(echo "$limit > $distance" | bc -l)
    if [ "$check" -eq 1 ]; then 
        counter=$(( counter + 1 ))
    fi
done

evilcounter=$(( $Nlines-$counter ))
echo "There are $counter entries whose distances are less than $limit, and $evilcounter entries whose distance are greater."
